In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# set defaults
plt.style.use('seaborn-white')   # seaborn custom plot style
plt.rc('figure', dpi=100, figsize=(7, 5))   # set default size/resolution
plt.rc('font', size=12)   # font size

tips = sns.load_dataset('tips')

# Examples

Outline

* Grid Search / Classification.
* Regression with feature selection (multicolinearity)
* Text model (bag of words /  tfidf).

### Example: Predicting Diabetes

* Given health measurements, can you predict whether an individual will develop diabetes?
* Classification of health outcomes: 0=NO DIABETES and 1=DIABETES
* We will use 'accuracy' to evaluate the model (not necessarily the best choice!)

In [ ]:
diabetes = pd.read_csv('data/diabetes.csv')
diabetes.head()

In [ ]:
diabetes[['Pregnancies','Insulin', 'BMI', 'Glucose', 'Outcome']].describe()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [ ]:
# features
X = diabetes.drop('Outcome', axis=1)
# outcome
y = diabetes.Outcome

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

In [ ]:
help(DecisionTreeClassifier)

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

In [ ]:
# proportion for correct values
clf.score(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
# look at the importance of each feature in our tree model
# _: attributes created when model is fit. Contains info that it learned from the data
dict(zip(X.columns, clf.feature_importances_))

In [ ]:
# maximum depth
clf.tree_.max_depth

In [ ]:
# node count
clf.tree_.node_count

In [ ]:
clf_small = DecisionTreeClassifier(max_depth=2)
clf_small.fit(X_train, y_train)
clf_small.score(X_train, y_train)

In [ ]:
clf_small.score(X_test, y_test)

In [ ]:
from util import tree_to_code

In [ ]:
tree_to_code(clf_small, X.columns)

In [ ]:
tree_to_code(clf, X.columns)

## How to select model parameters?

* Each combination of model parameters requires: 
    - fitting a model; evaluating the model; comparing performance.
* This leads to training perhaps *thousands* of models!
    - How to do this in a computationally feasible way?
    - How to keep your code clean to keep track?

## How to select model parameters?

* How to select in a computationally feasible way?
    - Trying every combination ('grid search') works for smaller data/models.
    - Sampling possible combinations of parameters using probabilistic reasoning.
    
* How to keep your code clean to keep track?
    - The `sklearn.model_selection` package.

### Example: parameter search for diabetes prediction

* Decision tree has possible parameters:
    - `max_depth`, `min_samples_split`, `min_samples_leaf`
    - plus others we won't consider
    
* We will use `GridSearchCV` to explore parameter combinations using cross-validation

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
# How many combinations are there to search through?

parameters = {
    'max_depth': [2,3,4,5,7,10,13,15,18,None], 
    'min_samples_split':[2,3,5,7,10,15,20],
    'min_samples_leaf':[2,3,5,7,10,15,20]
}

In [ ]:
clf = GridSearchCV(DecisionTreeClassifier(), parameters, cv = 5)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
clf.best_params_

In [ ]:
# number of models fit
len(clf.cv_results_['mean_test_score'])

### Was the "best parameter" the only "best choice"

* Plot the histogram of scores across all models
* Many high values implies many good model choices
    - Robust model
* Plot the histogram of standard deviations for the CVs
    - many small numbers imply most parameters give stable models.

In [ ]:
# Each x-value is the score of a different model,
# corresponding to a choice of parameters
#
plt.hist(clf.cv_results_['mean_test_score'])
plt.suptitle('accuracies on validation set for CV');

In [ ]:
# if most models were good and had high accuracy => good
# you do not want a model that really depends on parameter choices => likely, overfitting
# if a lot of good model choices: you are learning real patterns in the data 


In [ ]:
# Each x-value is the standard deviation for the 5-fold CV for a single model
# corresponding to a choice of parameters
plt.hist(clf.cv_results_['std_test_score'])
plt.suptitle('std deviation of CV scores');

In [ ]:
# if we have a high std of accuracy: model varied a lot, depending on how 
# you split your data.
# If std is low, then you are getting the same model performace 
# ^^ tells you what you should expect for fluctuation in your accuracy above


In [ ]:
clf_best = clf.best_estimator_

In [ ]:
clf.score(X_test, y_test)

In [ ]:
tree_to_code(clf_best, X.columns)

## Regression with Multicollinearity

* Linear regression with (perfectly) correlated features leads to high variance (unstable) models.
* When the dataset ~1-dimensional in 3-dim space, fitting a plane is under-determined.
* Regression gives you a plane of best fit but there are other planes that will give very similar error when fitting it that look very different: overfit model
* Use Principal Component Analysis to drop unneeded features.

<img src="imgs/multicollinearity.png" width="50%">

## Regression: predicting tips data

* Use quantitative feature "as is".
* Use all features: one-hot encode categorical features.
    - Many perfectly correlated features: e.g. `is_Lunch` and `is_Dinner`.

In [ ]:
tips = sns.load_dataset('tips')

In [ ]:
tips.head()

## Determining Correlated features

How many columns can we keep without losing information?

In [ ]:
catcols = ['sex', 'smoker', 'day', 'time']
pd.get_dummies(tips).head()      # only use get_dummies for illustration, NOT model building!

## Creating a Regression Pipeline
* Drop correlated features using Principal Component Analysis (PCA)
* Categorical Variables: One-Hot Encoding => PCA (drop correlated features)
* Quantitative Variables: Pass-through
* Estimator: Linear Regression

In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [ ]:
# svd_solver: how much information do you want to drop
# default: drops nothing
cats = Pipeline([
    ('ohe', OneHotEncoder(sparse=False)), 
    ('pca', PCA(svd_solver='full'))
])
catcols = ['sex', 'smoker', 'day', 'time']

nums = FunctionTransformer(lambda x:x)   # identity function
numcols = ['total_bill', 'size']

ct = ColumnTransformer([('cat', cats, catcols), ('num', nums, numcols)])

pl = Pipeline([('feats', ct), ('lr', LinearRegression())])


### Use Parameter Grid-Search with Cross-Validation

* Inspect the parameters of the pipeline with `get_params`: use to get keys for parameter-grid.
* Train models using a grid search and analyze the results!

In [ ]:
pl.get_params().keys()

In [ ]:
from sklearn.model_selection import GridSearchCV
# None: keep all variables
# 0.90 : keep 90% of the information
params = {'feats__cat__pca__n_components':[0.90, 0.99, None]}

# train using cross validation
grids = GridSearchCV(pl, param_grid=params, cv=5, iid = False)

In [ ]:
from sklearn.model_selection import train_test_split


X_tr, X_ts, y_tr, y_ts = train_test_split(tips, tips.tip)
grids.fit(X_tr, y_tr)

In [ ]:
grids.best_params_

In [ ]:
# The best_score_ is the average over your cross-validation fold scores of the best model 
# (best in exactly that sense: scores highest on average over folds)
grids.best_score_

In [ ]:
# Multicollinearity overfits a linear regression model
grids.cv_results_


In [ ]:
# score on the test set
grids.best_estimator_.score(X_ts, y_ts)

## Housing Price Prediction: Sales Price from House Characteristics

* Create a 'baseline' model: a model pipeline without ingenuity.
* Try to improve upon the the baseline with creativity and domain knowledge.

In [ ]:
housing = pd.read_csv('data/housing.csv')
housing.head(15)

In [ ]:
# drop id and SalesPrice from features

X = housing.drop(['Id', 'SalePrice'], axis=1)
y = housing['SalePrice']

## Baseline Model

* Do the 'minimum' to fit a model and check the performance.
    - Quantitative: impute with zero and pass-through (Why is zero not a *terrible* choice?)
    - Categorical: One-hot encoding (what assumption is this making about the data?)
    - Categorical: impute with string value "NULL"

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
types = X.dtypes
catcols = types.loc[types == np.object].index
numcols = types.loc[types != np.object].index

In [ ]:
cats = Pipeline([
    ('imp', SimpleImputer(strategy='constant', fill_value='NULL')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False)),
    ('pca', PCA(svd_solver='full', n_components=0.99))
])

ct = ColumnTransformer([
    ('catcols', cats, catcols),
    ('numcols', SimpleImputer(strategy='constant', fill_value=0), numcols)
])

pl = Pipeline([('feats', ct), ('reg', LinearRegression())])

In [ ]:
X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.25)

In [ ]:
# R^2
# It provides an indication of goodness of fit and therefore a measure 
# of how well unseen samples are likely to be predicted by the model, 
# through the proportion of explained variance.

# https://scikit-learn.org/stable/modules/model_evaluation.html#r2-score

# Possible negative value
# That it's worse than a constant model (that guessed the average)! 
# There's more variance in the predictions than in the true data.


pl.fit(X_tr, y_tr)
pl.score(X_ts, y_ts)

In [ ]:
# RMSE
preds = pl.predict(X_ts)
np.sqrt(np.mean(preds - y_ts)**2)

In [ ]:
out = []
for _ in range(100):
    X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.25)
    pl.fit(X_tr, y_tr)
    out.append(pl.score(X_ts, y_ts))

In [ ]:
pd.Series(out).plot(kind='hist', title='scores in 100 model builds');

## Improved Model

Question: What features and model improvements can we use?

In [ ]:
housing.head()

## Improved Model

* If using linear regression: PCA to drop extra information.
* Better techniques for categorical encoding? (Ordinal vs Nominal?)
* Better techniques for imputation?
* Other models to try? (Tree-based?)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

cats = Pipeline([
    ('imp', SimpleImputer(strategy='constant', fill_value='NULL')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False)),
    ('pca', PCA(svd_solver='full', n_components=0.99))
])

ct = ColumnTransformer([
    ('catcols', cats, catcols),
    ('numcols', SimpleImputer(strategy='constant', fill_value=0), numcols)
])

pl = Pipeline([('feats', ct), ('reg', RandomForestRegressor())])

In [ ]:
X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.25)

In [ ]:
pl.fit(X_tr, y_tr)
pl.score(X_ts, y_ts)

In [ ]:
preds = pl.predict(X_ts)
np.sqrt(np.mean(preds - y_ts)**2)

In [ ]:
out = []
for _ in range(100):
    X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.25)
    pl.fit(X_tr, y_tr)
    out.append(pl.score(X_ts, y_ts))

In [ ]:
pd.Series(out).plot(kind='hist', title='scores in 100 model builds');

## Predicting Reviews

In [ ]:
#reviews = pd.read_json(open('../../labs/lab09/data/reviews.json'), lines=True)
reviews = pd.read_json(open('data/reviews.json'), lines=True)

In [ ]:
reviews.head()

In [ ]:
# binary classifier
# > 3 stars: 1
X = reviews['summary']
y = (reviews[['overall']] > 3).astype(int)

X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.25)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

# Take a decision tree model, train it once
# Train a, say, 100 decision tree models, average their result
# cuts down on overfitting

In [ ]:
count_vec = CountVectorizer()  # bag of words vectors on text fields

In [ ]:
count_vec.fit(X_tr)

In [ ]:
# vocabulary 
vocab = sorted(count_vec.vocabulary_, key=lambda x:x[1])

pd.DataFrame(
    data=count_vec.transform(X).toarray(),
    columns=vocab
).head()

In [ ]:
# building random forest on the features 
pl = Pipeline([
    ('cv', CountVectorizer()), 
    ('clf', RandomForestClassifier(max_depth=8, n_estimators=7))
])

In [ ]:
# fit a pipeline
pl.fit(X_tr, y_tr)

In [ ]:
# score it
pl.score(X_ts, y_ts)

In [ ]:
clf = pl.named_steps['clf']
clf

In [ ]:
# What are the most important features?
pd.Series(
    data=clf.feature_importances_,
    index=vocab
).sort_values(ascending=False)

In [ ]:
word = 'annoy'

reviews[reviews['summary'].str.lower().str.contains(word)]

In [ ]:
reviews[reviews['summary'].str.lower().str.contains(word)].overall.plot(kind='hist')

In [ ]:
pl.get_params().keys()

In [ ]:
params = {
    'clf__max_depth': np.arange(2,500,20)
}

In [ ]:
grids = GridSearchCV(pl, param_grid=params, cv=3)

In [ ]:
grids.fit(X_tr, y_tr)

In [ ]:
index = grids.param_grid['clf__max_depth']
test = grids.cv_results_['mean_test_score']
train = grids.cv_results_['mean_train_score']

In [ ]:
pd.DataFrame({'test': test, 'train': train}, index=index).plot()

In [ ]:
params = {
    'clf__n_estimators': [2, 5, 10, 25, 100],
    'clf__max_depth': [2, 5, 9, None],
    'cv__max_features': [100, None], 
    'cv__strip_accents': [None, 'unicode']
}

In [ ]:
grids = GridSearchCV(pl, param_grid=params, cv=3)

In [ ]:
grids.fit(X_tr, y_tr)

In [ ]:
grids.best_params_